In [1]:
!pip install --upgrade transformers
!pip install accelerate

# GPTQ Dependencies
!pip install --upgrade optimum
!pip install --upgrade auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys, json

In [3]:
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
    )

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# Create a pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text = False  # Only return the current output instead of returning complete prompt
    )

In [18]:
def parse_first_json(json_str):
    end_index = json_str.find("}") # Find the index of the first closing curly brace
    if end_index != -1:  # If a closing brace is found
        first_json_obj = json_str[:end_index + 1] # Parse the substring containing the first JSON object
        parsed_json = json.loads(first_json_obj) # Parse the JSON object
        return parsed_json
    else:
        return None

role = "an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies."
story = ""

for _ in range(3):
  user = f"""Starting from now, assume the role of {role}.
  Generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets.

  [{story}]

  Also give 3 choices, that is, 3 possible continuations of the story. Every time you must return only a JSON object with a single entry with the follwoing two keys strictly following the provided description:
  {{
  "story": string, the generated narrative.
  "options": list, a list containing three string-type elements corresponding to the 3 options.
  }}
  Do not generate anything other than the JSON object inside {{}}.
  """

  chat = [
    {"role": "user", "content": user}
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False)

  outputs = pipe(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
        )
  output = outputs[0]["generated_text"].strip()

  data = parse_first_json(output)

  narrative = data["story"]
  options = data["options"]
  print(narrative)
  [print(opt) for opt in options]
  user_input = input("> ")
  story += data["story"] + user_input

As the sun began to set, the ancient ruins came alive with the sound of chirping crickets and the smell of damp earth. Suddenly, a rustling in the bushes revealed a hidden entrance. Do you: 1) Enter the ruins, 2) Ignore it and continue on your journey, or 3) Set up camp and explore the area further?
Enter the ruins
Ignore it and continue on your journey
Set up camp and explore the area further
> Set up camp and explore the area further
The sound of crickets grew louder as you set up camp. The damp earth smelled of history and secrets. As twilight deepened, a glowing ember revealed a hidden treasure map. Do you: 1) Follow the map into the ruins, 2) Keep the map as a souvenir and continue on your journey, or 3) Share the map with your companions?
1) Follow the map into the ruins
2) Keep the map as a souvenir and continue on your journey
3) Share the map with your companions
> Share the map with your companions
The glowing ember revealed a hidden treasure map, but as you shared it with yo

In [17]:
print(prompt)

<s>[INST] Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies..
  Generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets.

  []

  Also give 3 choices, that is, 3 possible continuations of the story. Every time you must return only a  JSON object with a single entry  with the follwoing two keys strictly following the provided description:
  {
  "story": string, the generated narrative.
  "options": list, a list containing three string-type elements corresponding to the 3 options.
  }
  Do not generate anything other than the JSON object inside {}.
   [/INST]
  {'story': "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.",
   'options': ['Seek help from the village elder.',
               'Search for a way to break the curse.',
    

In [19]:
print(story)

As the sun began to set, the ancient ruins came alive with the sound of chirping crickets and the smell of damp earth. Suddenly, a rustling in the bushes revealed a hidden entrance. Do you: 1) Enter the ruins, 2) Ignore it and continue on your journey, or 3) Set up camp and explore the area further?Set up camp and explore the area furtherThe sound of crickets grew louder as you set up camp. The damp earth smelled of history and secrets. As twilight deepened, a glowing ember revealed a hidden treasure map. Do you: 1) Follow the map into the ruins, 2) Keep the map as a souvenir and continue on your journey, or 3) Share the map with your companions?Share the map with your companionsThe glowing ember revealed a hidden treasure map, but as you shared it with your companions, they became uneasy. One suggested selling it, another hoarding it, and a third was hesitant, fearing the consequences. Do you: 1) Persuade them to follow the map, 2) Keep the map and continue on your own, or 3) Leave th